In [ ]:
# Checklist...
#
# Make sure units have been converted in fisher matrix to match units in mcmc chains (use 'Fisher_Units_Converter' notebook.)
#
# Make sure parameters in fisher matrix match order of parameter in MCMC
#
# Make sure you burn in the right number of points in the MCMC chains before plotting below 
#
# Choose where to plot Fisher contour centers (where you forecasted [correct choice], or max likelihood of MCMCs)
#
#

# FORECAST/MCMC MATCH

In [ ]:
# Show plots inline, and load main getdist plot module and samples class
from __future__ import print_function
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import sys, os
sys.path.insert(0,os.path.realpath(os.path.join(os.getcwd(),'..')))
from getdist import plots, MCSamples, loadMCSamples
from getdist.gaussian_mixtures import GaussianND
import getdist
# use this *after* importing getdist if you want to use interactive plots
# %matplotlib notebook
import matplotlib.pyplot as plt
import IPython
import numpy as np
import pandas as pd
print('GetDist Version: %s, Matplotlib version: %s'%(getdist.__version__, plt.matplotlib.__version__))
# matplotlib 2 may not work very well without usetex on, can uncomment
# plt.rcParams['text.usetex']=True

In [ ]:
# First run montepython to determine the burn-in period 
# Specify either number of lines (e.g. 100) or proportion (e.g. 0.3)
yesstep_burnin = 100
nostep_burnin = 100

In [ ]:
#Import MCMC Chains

chains_dir = "/Users/nicholasdeporzio/Desktop/cfworkspace/chains/"
yesstep_dir = chains_dir + "chains_yes_relicfast/Relicfast_CMBS4_desi_DH/2020-01-30_1000000_"
nostep_dir = chains_dir + "chains_no_relicfast/CMBS4_desi_DH/2020-01-30_1000000_"

samples_yesstep = loadMCSamples(yesstep_dir, no_cache=True, settings={'ignore_rows':yesstep_burnin})
samples_nostep = loadMCSamples(nostep_dir, no_cache=True, settings={'ignore_rows':nostep_burnin})

In [ ]:
# Triangle plot, no Fisher Contours
g = plots.get_subplot_plotter()

# Set a default plot style 
from getdist.styles.tab10 import style_name
plots.set_active_style(style_name)

# Adjust plot sizes, styles, etc using getdist.plots.GetDistPlotSettings class
g.settings.figure_legend_frame = False #Box around legend
g.settings.scaling=False #Defaults to font size for Tick Labels: 11, Labels: 14, Legend: 12, Text: 12
g.settings.axes_fontsize=14
g.settings.axes_labelsize=20
g.settings.axis_tick_x_rotation=45
g.settings.legend_fontsize=20
#g.settings.axis_tick_y_rotation=90
#g.settings.colorbar_tick_rotation=90

g.triangle_plot(
    [samples_yesstep, samples_nostep], 
    filled=True,
    legend_labels=['MCMC, DESI, Relic/LCDM Steps ON', 'MCMC, DESI, Relic/LCDM Steps OFF']
    )
plt.savefig("/Users/nicholasdeporzio/Desktop/only_mcmc.png")

In [ ]:
# Add Fisher Contours
# Need to adjust Fisher parameter ordering to MCMC parameter ordering
# DESI Fisher parameter ordering: omega_b, omega_cdm, n_s, A_s, tau_reio, h, m_0, sigma_fog, b0_bar, alpha_k2
# DESI MCMC parameter ordering: omega_b, omega_cdm, n_s, A_s, h, tau_reio, m_0, sigma_fog, b0_bar, alpha_k2

fisher_reorder_index = [0, 1, 2, 3, 5, 4, 6, 7, 8, 9] 

#yesstep_covariance_in = np.loadtxt("/Users/nicholasdeporzio/Desktop/cfworkspace/results/ps2/Dynamic_KMax/FidOnForecastedYesStepShift_CMBS4_desi_DH/inv_fisher.mat", skiprows=1)
yesstep_covariance_in = np.loadtxt("/Users/nicholasdeporzio/Desktop/cfworkspace/results/ps2/Dynamic_KMax/FidOnYesStepBestfit_CMBS4_desi_DH/inv_fisher.mat", skiprows=1)
nostep_covariance_in = np.loadtxt("/Users/nicholasdeporzio/Desktop/cfworkspace/results/ps2/Dynamic_KMax/FidOnNoStepBestfit_CMBS4_desi_DH/inv_fisher.mat", skiprows=1)

yesstep_covariance_out = np.zeros((10,10))
nostep_covariance_out = np.zeros((10,10))

for idx1 in range(10): 
    for idx2 in range(10): 
        yesstep_covariance_out[idx1, idx2] = yesstep_covariance_in[fisher_reorder_index[idx1], fisher_reorder_index[idx2]]
        nostep_covariance_out[idx1, idx2] = nostep_covariance_in[fisher_reorder_index[idx1], fisher_reorder_index[idx2]]
        
        

In [ ]:
# Bestfit for Yes Relicfast Relicfast_CMBS4_desi_DH chains
mcmc_yesstep_bestfit = [
    2.224689e-02 * 1.0e2,
    1.124890e-01,
    9.670359e-01,
    2.240779e-09 * 1.0e9,
    7.014706e-01,
    6.196884e-02,
    3.639409e-02,
    2.502574e+02,
    9.993837e-01,
    1.042483e+00
]

# Bestfit for No Relicfast CMBS4_desi_DH chains
mcmc_nostep_bestfit = [
    2.227047e-02 * 1.0e2, 
    1.127401e-01, 
    9.659302e-01, 
    2.234591e-09 * 1.0e9, 
    7.007968e-01, 
    6.019503e-02, 
    3.623808e-02, 
    2.497568e+02, 
    1.003223e+00, 
    9.948011e-01
]

# Result of forecasted shift due to turning relicfast step on starting at No relicfast fiducial cosmology
forecast_yesstep_shift = [
    (2.227047e-02+1.1602668787041021e-06)*1.0e2, 
    1.127401e-01-4.108796815856762e-05, 
    9.659302e-01+5.311035111640709e-05, 
    (2.234591e-09-4.806938031745625e-12)*1.0e9, 
    7.007968e-01+0.0010625065009387986, 
    6.019503e-02-0.001064158458315645, 
    3.623808e-02-(0.009823889445796/3), 
    2.497568e+02+(762.6869526527007*1.0e-3), 
    1.003223e+00-0.004980685985104448, 
    9.948011e-01+0.10341895410597579
]



In [ ]:
paramnames = [r'omega_b', r'omega_cdm', r'n_s', r'A_s', r'h', r'tau_reio', r'm_0', r'sigma_fog', r'b0bar', r'alpha_k2']
#yesstep_gauss = GaussianND(forecast_yesstep_shift, yesstep_covariance_out, names=paramnames)
yesstep_gauss = GaussianND(mcmc_yesstep_bestfit, yesstep_covariance_out, names=paramnames)
nostep_gauss = GaussianND(mcmc_nostep_bestfit, nostep_covariance_out, names=paramnames)

h = plots.get_subplot_plotter()

# Set a default plot style 
from getdist.styles.tab10 import style_name
plots.set_active_style(style_name)

# Adjust plot sizes, styles, etc using getdist.plots.GetDistPlotSettings class
h.settings.figure_legend_frame = False #Box around legend
h.settings.scaling=False #Defaults to font size for Tick Labels: 11, Labels: 14, Legend: 12, Text: 12
h.settings.axes_fontsize=14
h.settings.axes_labelsize=20
h.settings.axis_tick_x_rotation=45
h.settings.legend_fontsize=20
#h.settings.axis_tick_y_rotation=90
#h.settings.colorbar_tick_rotation=90

h.triangle_plot(
    [yesstep_gauss, nostep_gauss], 
    filled=True,
    legend_labels=['Fisher, DESI, Relic/LCDM Steps ON', 'Fisher, DESI, RELIC/LCDM Steps OFF'])
plt.savefig("/Users/nicholasdeporzio/Desktop/only_fisher.png")

In [ ]:
i = plots.get_subplot_plotter()

# Set a default plot style 
from getdist.styles.tab10 import style_name
plots.set_active_style(style_name)

# Adjust plot sizes, styles, etc using getdist.plots.GetDistPlotSettings class
i.settings.figure_legend_frame = False #Box around legend
i.settings.scaling=False #Defaults to font size for Tick Labels: 11, Labels: 14, Legend: 12, Text: 12
i.settings.axes_fontsize=14
i.settings.axes_labelsize=20
i.settings.axis_tick_x_rotation=45
i.settings.legend_fontsize=20
#i.settings.axis_tick_y_rotation=90
#i.settings.colorbar_tick_rotation=90

paramnames = ['omega_b', 'omega_cdm', 'n_s', 'A_s', 'h', 'tau_reio', 'm_0', 'sigma_fog', 'b0bar', 'alpha_k2']
#yesstep_gauss = GaussianND(forecast_yesstep_shift, yesstep_covariance_out, names=paramnames)
yesstep_gauss = GaussianND(mcmc_yesstep_bestfit, yesstep_covariance_out, names=paramnames)
nostep_gauss = GaussianND(mcmc_nostep_bestfit, nostep_covariance_out, names=paramnames)

i.triangle_plot(
    [samples_yesstep, samples_nostep, yesstep_gauss, nostep_gauss], 
    legend_labels=['MCMC, DESI + CMBS4, Relic/LCDM Steps ON', 'MCMC, DESI + CMBS4, Relic/LCDM Steps OFF', 'Fisher, DESI + CMBS4, Relic/LCDM Steps ON', 'Fisher, DESI + CMBS4, Relic/LCDM Steps OFF'],
    line_args=[
        {'color':'green'},
        {'color':'darkblue'},
        {'ls':'dotted', 'lw':2, 'color':'#A51C30'},
        {'ls':'--', 'lw':2, 'color':'#93a1ad'}], 
    contour_args=[
        {'color':'green', 'filled' : True},
        {'color':'darkblue', 'filled' : True},
        {'ls':'dotted', 'lw':2, 'color':'#A51C30', 'filled' : False},
        {'ls':'--', 'lw':2, 'color':'#93a1ad', 'filled' : False}]
)

plt.savefig("/Users/nicholasdeporzio/Desktop/mcmc_and_fisher.png")

# NEUTRINO/EQUIVALENT RELIC MATCH

In [ ]:
# Show plots inline, and load main getdist plot module and samples class
from __future__ import print_function
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import sys, os
sys.path.insert(0,os.path.realpath(os.path.join(os.getcwd(),'..')))
from getdist import plots, MCSamples, loadMCSamples
from getdist.gaussian_mixtures import GaussianND
import getdist
# use this *after* importing getdist if you want to use interactive plots
# %matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')
import IPython
import numpy as np
import pandas as pd
print('GetDist Version: %s, Matplotlib version: %s'%(getdist.__version__, plt.matplotlib.__version__))
# matplotlib 2 may not work very well without usetex on, can uncomment
plt.rcParams['text.usetex']=True

# Full parameter set
#paramnames = [r'$\omega_b$', r'$\omega_{cdm}$', r'$n_s$', r'$A_s$', r'$\tau_{reio}$', 'h', r'$M_\chi$', r'$\sigma_{fog}$', r'$\bar{b}_0$', r'$\alpha_{k2}$']
#fiducials = [2.222, 0.1120, 0.965, 2.22, 0.06, 0.7, 0.09, 250, 1.0, 1.0]

# Reduced parameter set
paramnames = [r'$\omega_b$', r'$\omega_{cdm}$', r'$n_s$', r'$10^{9}A_s$', r'$\tau_{reio}$', '$h$', r'$M_\chi$']
fiducials = [2.222, 0.1120, 0.965, 2.22, 0.06, 0.7, 0.09]

reliccovmat = np.loadtxt("/Users/nicholasdeporzio/Desktop/cfworkspace/results/old_results/relicneutrino_match/relic/inv_fullfisher.mat", skiprows=1)
neutrinocovmat = np.loadtxt("/Users/nicholasdeporzio/Desktop/cfworkspace/results/old_results/relicneutrino_match/neutrino/inv_fullfisher.mat", skiprows=1)

relicgauss = GaussianND(fiducials, reliccovmat[0:-3, 0:-3], names=paramnames)
neutrinogauss = GaussianND(fiducials, neutrinocovmat[0:-3, 0:-3], names=paramnames)

g = plots.get_subplot_plotter()

# Set a default plot style 
from getdist.styles.tab10 import style_name
plots.set_active_style(style_name)

# Adjust plot sizes, styles, etc using getdist.plots.GetDistPlotSettings class
g.settings.figure_legend_frame = False #Box around legend
g.settings.scaling=False #Defaults to font size for Tick Labels: 11, Labels: 14, Legend: 12, Text: 12
#g.settings.scaling_factor = 0.3 #Scaling of font size
#g.settings.title_limit_fontsize = 14
g.settings.axes_fontsize=14
g.settings.axes_labelsize=20
g.settings.axis_tick_x_rotation=45
g.settings.legend_fontsize=20
#g.settings.axis_tick_y_rotation=90
#g.settings.colorbar_tick_rotation=90

g.triangle_plot(
    [relicgauss, neutrinogauss], 
    #filled=True,
    legend_labels=['Thermal Relic', 'Equivalent Neutrino'],
    line_args = [
        {'ls':'--', 'lw':2, 'color':'red'},
        {'ls':'--', 'lw':2, 'color':'grey'}],
    contour_args=[
        {'ls':'--', 'lw':2, 'color':'red', 'filled' : False},
        {'ls':'--', 'lw':2, 'color':'grey', 'filled' : False}])
plt.savefig("/Users/nicholasdeporzio/Desktop/relicneutrino_match.png")